In [13]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import EulerDiscreteScheduler # Import samplers
from diffusers import DPMSolverMultistepScheduler # Import samplers
from diffusers.utils import load_image
from tqdm import tqdm
# Prompt generation
from prompt import prompt_generator, prompt_generator_no_human

Samplers (schedulers): https://huggingface.co/docs/diffusers/api/schedulers/overview

In [2]:
print("Torch version:",torch.__version__)
print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.2.2+cu121
Is CUDA enabled? True


https://thepythoncode.com/article/control-generated-images-with-controlnet-with-huggingface#code

# First Pipeline: ControlNet + Stable Diffusion 

In [8]:
# ControlNet model
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble", torch_dtype=torch.float16)

# Define stable diffusion pipeline with controlnet
# We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
pipe = StableDiffusionControlNetPipeline.from_pretrained("stablediffusionapi/realistic-vision-v20-2047", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

# Enable efficient implementations using xformers for faster inference
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

unet\diffusion_pytorch_model.safetensors not found
Loading pipeline components...: 100%|██████████| 6/6 [00:11<00:00,  1.97s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [22]:
# Load wireframe image
image_input = load_image("wireframes_set1\\img3_4.jpg")

# Prompt
prompt = prompt_generator_no_human()
print("Prompt:",prompt)
neg_prompt = "multiple boxes in the background, cardboard, a person in the foreground, not realistic"
# Run the pipeline
image_output = pipe(prompt=prompt, negative_prompt=neg_prompt, image=image_input, num_inference_steps=30).images[0]

# Save the output
image_output.save("test.jpg")
print("Save test.jpg!")

Prompt: A medium shot view of a colorful cardboard box with red tape from the Bol.com webshop, in front of a quite neighborhood street and cars background, the weather is sunny, high photorealistic quality.


100%|██████████| 30/30 [00:15<00:00,  1.94it/s]


Save test.jpg!


In [23]:
prompt = prompt_generator()
print(prompt)

A mid shot view of an angry man holding a white cardboard box with red tape from the Adidas webshop in his hands, standing in front of a garbage curb background, the weather is cloudy, high photorealistic quality.


# Generating first synthetic dataset

wireframes_set1 are wireframes drawn on the bottom half region of the window, so they can easily be held  by persons.
Next, we will generate synthetic images of those wireframes in the directory output_set1.
- I generate random prompts
- Next I will validate them
- Finally, I will only keep the images with a single box in the image

ControlNet model: lllyasviel/control_v11p_sd15_scribble"
Stable diffusion model: stablediffusionapi/realistic-vision-v20-2047

In [26]:
import glob
import os
from prompt import prompt_generator

def generate_images(directory_path, output_directory_path, controlnet_model_path, sd_model_path, prompt_human):
    
    directory_path = directory_path
    output_directory_path = output_directory_path
    
    # ControlNet model
    controlnet = ControlNetModel.from_pretrained(controlnet_model_path, torch_dtype=torch.float16)

    # Define stable diffusion pipeline with controlnet
    # We use the realistic-vision-v20-2047 model for this example (finetunned on realistic images of people)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(sd_model_path, controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16)
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

    # Enable efficient implementations using xformers for faster inference
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_model_cpu_offload()
    

    for file in tqdm(glob.glob(os.path.join(directory_path, '*.jpg'))):
        image_input = load_image(file)
        prompt = prompt_generator_no_human() if prompt_human == 0 else prompt_generator()
        print(prompt)
        image_output = pipe(prompt, image_input, num_inference_steps=30).images[0]
        image_output.save(os.path.join(output_directory_path, os.path.basename(file)[3:]))

In [27]:
generate_images("wireframes_set1", "output_set1", "lllyasviel/control_v11p_sd15_scribble", "stablediffusionapi/realistic-vision-v20-2047", 1)

unet\diffusion_pytorch_model.safetensors not found
Loading pipeline components...:  33%|███▎      | 2/6 [00:04<00:08,  2.24s/it]c:\Users\aaron\Documents\git\controlnet\.myvenv\Lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:13<00:00,  2.30s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve anal

A close-up view of a brown cardboard box with green tape from the H&M webshop, in front of a quite neighborhood street and garden background, the weather is rainy, high photorealistic quality.


  4%|▍         | 1/25 [00:20<08:06, 20.26s/it]

A close-up view of a colorful cardboard box with black tape from the Amazon webshop, in front of a garbage curb background, the weather is snowy, high photorealistic quality.


  8%|▊         | 2/25 [00:39<07:27, 19.47s/it]

...

A high angle view of a colorful cardboard box with yellow tape from the Zara webshop, in front of a garbage curb background, the weather is sunny, high photorealistic quality.


 88%|████████▊ | 22/25 [06:41<00:55, 18.59s/it]

A high angle view of a brown cardboard box with green tape from the Zalando webshop, in front of a garbage curb background, the weather is sunny, high photorealistic quality.


 92%|█████████▏| 23/25 [06:59<00:37, 18.61s/it]

A medium shot view of a brown cardboard box with a barcode from the Coolblue webshop, in front of a garbage curb background, the weather is snowy, high photorealistic quality.
